In [26]:
pip install Sastrawi


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [27]:
!pip install Sastrawi gensim


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Using cached gensim-4.4.0-cp310-cp310-win_amd64.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   -- ------------------------------------- 1.6/24.4 MB 10.5 MB/s eta 0:00:03
   ----- ---------------------------------- 3.4/24.4 MB 9.6 MB/s eta 0:00:03
   --------- ------------------------------ 5.5/24.4 MB 10.2 MB/s eta 0:00:02
   ------------ --------------------------- 7.9/24.4 MB 10.1 MB/s eta 0:00:02
   ----------------- ---------------------- 10.5/24.4 MB 10.6 MB/s eta 0:00:02
   --------------------- ------------------ 13.1/24.4 MB 11.0 MB/s eta 0:00:02
   ------------------------ --------------- 15.2/24.4 MB 11.1 MB/s eta 0:00:01
   ----------------------------- ---------- 17.8/24.4 MB 11.0 MB/s eta 0:00:01
   -------------------------------- ------- 19.7/24.4 MB 10.7 MB/s eta 0:00:01
   ---------------------------------- ----- 21.0/24.4 MB 10.4 MB/s eta 0:00:01
   ------------------------------------- -- 22.8/24.4 MB 10.1 MB/s eta 0

In [ ]:
# ============================================================
# IMPORT LIBRARY
# ============================================================
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler 
from gensim.models import Word2Vec 
from sklearn.ensemble import RandomForestClassifier 

# Preprocessing
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Models
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

# ============================================================
# KONFIGURASI DATASET
# ============================================================
RANDOM_STATE = 42
CSV_PATH = "dataset_manual_label.csv"

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [31]:
df = pd.read_csv(CSV_PATH, encoding='latin-1', sep=';')

print("Jumlah data:", len(df))
df.head()

Jumlah data: 250


,text,text_cleaned_full,manual
0,kepo bgt adekkkk??????????????,kepo adekkkk,negatif
1,"buah jatuh se pohon "" nya????",buah jatuh se pohon nya,negatif
2,Cushion Nya kegelapan ga sihhh??,cushion nya kegelapan sihhh,negatif
3,LU APAIN PONI BOCIL ITU KAK JENNN?!!! KENAPA L...,lu apain poni bocil kak jennn kenapa lu buat s...,negatif
4,mayi gamau kalah cantik sama mamanya????,mayi gamau kalah cantik sama mamanya,negatif


In [32]:
# Setup stopwords, stemmer, dan fungsi preprocess (stemm + simple lemmatize)

# Stopwords & stemmer Sastrawi
stopword_factory = StopWordRemoverFactory()
STOP_WORDS = set(stopword_factory.get_stop_words())

stemmer = StemmerFactory().create_stemmer()

# Simple lemmatization dictionary (tambahkan sesuai kebutuhan dataset)
LEMMATIZE_DICT = {
    "kegabutan": "gabut",
    "kegelapan": "gelap",
    "gamau": "tidak mau",
    "kepo": "ingin tahu",
    "poni": "poni", 
    "kak": "kakak",
    "adekkkk": "adik",
}

def preprocess_text(text):
    """
    Steps:
    - safe check NA
    - lowercase, remove non-letters (retain spaces)
    - tokenize by whitespace
    - remove stopwords
    - sastrawi stem per token
    - simple lemmatize by dict
    - join back
    """
    if pd.isna(text):
        return ""
    s = str(text).lower()
    s = re.sub(r'[^a-z\s]', ' ', s)  # hanya huruf a-z (indonesia), angka/punctuation -> space
    toks = s.split()
    toks = [t for t in toks if t and t not in STOP_WORDS]
    # stem each token
    stemmed = [stemmer.stem(t) for t in toks]
    # lemmatize via dictionary fallback
    lemm = [LEMMATIZE_DICT.get(t, t) for t in stemmed]
    return " ".join(lemm)


In [ ]:
# --- LOAD DATA ---
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"File tidak ditemukan: {CSV_PATH}")

df = pd.read_csv(CSV_PATH, encoding='latin-1', sep=';')

print("=== PREVIEW DATASET ===")
display(df.head())

print("\n=== DISTRIBUSI LABEL ===")
print(df["manual"].value_counts(dropna=False))

=== PREVIEW DATASET ===


,text,text_cleaned_full,manual
0,kepo bgt adekkkk??????????????,kepo adekkkk,negatif
1,"buah jatuh se pohon "" nya????",buah jatuh se pohon nya,negatif
2,Cushion Nya kegelapan ga sihhh??,cushion nya kegelapan sihhh,negatif
3,LU APAIN PONI BOCIL ITU KAK JENNN?!!! KENAPA L...,lu apain poni bocil kak jennn kenapa lu buat s...,negatif
4,mayi gamau kalah cantik sama mamanya????,mayi gamau kalah cantik sama mamanya,negatif



=== DISTRIBUSI LABEL ===
manual
negatif    100
positif    100
netral      50
Name: count, dtype: int64


In [ ]:
possible_cols = ["text_cleaned_full", "clean_text", "manual", "text"]

text_col = None
for col in possible_cols:
    if col in df.columns:
        text_col = col
        break

if text_col is None:
    raise ValueError(f"Tidak ada kolom teks. Minimal salah satu kolom harus ada: {possible_cols}")

print(f"\nMenggunakan kolom teks: {text_col}")

# --- PERSIAPAN TEKS & LABEL ---
df = df.dropna(subset=["manual"]).reset_index(drop=True)
texts_raw  = df[text_col].astype(str)
labels_raw = df["manual"].astype(str)


print("\n[INFO] Melakukan preprocessing (stem + lemmatization)...")
texts_processed = texts_raw.apply(preprocess_text)

# --- TRAIN-TEST SPLIT ---
try:
    X_train, X_test, y_train, y_test = train_test_split(
        texts_processed,
        labels_raw,
        test_size=0.2,
        random_state=RANDOM_STATE,
        stratify=labels_raw
    )
except ValueError:
    print("[WARNING] Stratify gagal (kelas terlalu sedikit). Melakukan split tanpa stratify.")
    X_train, X_test, y_train, y_test = train_test_split(
        texts_processed,
        labels_raw,
        test_size=0.2,
        random_state=RANDOM_STATE
    )

print("\n=== DISTRIBUSI LABEL SETELAH SPLIT ===")
print("Train:", dict(Counter(y_train)))
print("Test :", dict(Counter(y_test)))


Menggunakan kolom teks: text_cleaned_full

[INFO] Melakukan preprocessing (stem + lemmatization)...

=== DISTRIBUSI LABEL SETELAH SPLIT ===
Train: {'negatif': 80, 'netral': 40, 'positif': 80}
Test : {'positif': 20, 'negatif': 20, 'netral': 10}


In [36]:
# TF-IDF + LinearSVC
print("=== Percobaan 1: TF-IDF + LinearSVC ===")
pipe1 = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=15000, ngram_range=(1,2), sublinear_tf=True, min_df=2)),
    ("svc", LinearSVC(class_weight="balanced", random_state=RANDOM_STATE))
])

pipe1.fit(X_train, y_train)
pred1 = pipe1.predict(X_test)

acc1 = accuracy_score(y_test, pred1)
print(f"Akurasi: {acc1:.4f}\n")
print("Classification Report:")
print(classification_report(y_test, pred1, zero_division=0))


=== Percobaan 1: TF-IDF + LinearSVC ===
Akurasi: 0.5000

Classification Report:
              precision    recall  f1-score   support

     negatif       0.50      0.40      0.44        20
      netral       0.11      0.10      0.11        10
     positif       0.64      0.80      0.71        20

    accuracy                           0.50        50
   macro avg       0.42      0.43      0.42        50
weighted avg       0.48      0.50      0.48        50



In [37]:
# BOW + MultinomialNB
print("=== Percobaan 2: BOW (CountVectorizer) + MultinomialNB ===")
pipe2 = Pipeline([
    ("bow", CountVectorizer(max_features=15000, ngram_range=(1,2), min_df=2)),
    ("nb", MultinomialNB())
])

pipe2.fit(X_train, y_train)
pred2 = pipe2.predict(X_test)

acc2 = accuracy_score(y_test, pred2)
print(f"Akurasi: {acc2:.4f}\n")
print("Classification Report:")
print(classification_report(y_test, pred2, zero_division=0))


=== Percobaan 2: BOW (CountVectorizer) + MultinomialNB ===
Akurasi: 0.5400

Classification Report:
              precision    recall  f1-score   support

     negatif       0.54      0.70      0.61        20
      netral       0.33      0.10      0.15        10
     positif       0.57      0.60      0.59        20

    accuracy                           0.54        50
   macro avg       0.48      0.47      0.45        50
weighted avg       0.51      0.54      0.51        50



In [38]:
# Word2Vec + average embeddings + LogisticRegression
print("=== Percobaan 3: Word2Vec(avg) + LogisticRegression ===")

def tokenize_list(texts):
    return [t.split() for t in texts]

sentences = tokenize_list(X_train.tolist())
w2v_size = 100
w2v = Word2Vec(sentences, vector_size=w2v_size, window=5, min_count=1, workers=4, seed=RANDOM_STATE, epochs=20)

def avg_vector(text, model, size=w2v_size):
    toks = text.split()
    vecs = [model.wv[t] for t in toks if t in model.wv]
    if len(vecs) == 0:
        return np.zeros(size)
    return np.mean(vecs, axis=0)

X_train_vec = np.vstack([avg_vector(t, w2v) for t in X_train])
X_test_vec  = np.vstack([avg_vector(t, w2v) for t in X_test])

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train_vec)
X_test_s  = scaler.transform(X_test_vec)

lr = LogisticRegression(max_iter=1000, class_weight="balanced", random_state=RANDOM_STATE)
lr.fit(X_train_s, y_train)
pred3 = lr.predict(X_test_s)

acc3 = accuracy_score(y_test, pred3)
print(f"Akurasi: {acc3:.4f}\n")
print("Classification Report:")
print(classification_report(y_test, pred3, zero_division=0))


=== Percobaan 3: Word2Vec(avg) + LogisticRegression ===
Akurasi: 0.4600

Classification Report:
              precision    recall  f1-score   support

     negatif       0.50      0.55      0.52        20
      netral       0.00      0.00      0.00        10
     positif       0.55      0.60      0.57        20

    accuracy                           0.46        50
   macro avg       0.35      0.38      0.37        50
weighted avg       0.42      0.46      0.44        50



In [39]:
# TF-IDF + RandomForest
print("=== Percobaan 4: TF-IDF + RandomForest ===")
pipe4 = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=10000, ngram_range=(1,2), min_df=2)),
    ("rf", RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1, class_weight="balanced_subsample"))
])

pipe4.fit(X_train, y_train)
pred4 = pipe4.predict(X_test)

acc4 = accuracy_score(y_test, pred4)
print(f"Akurasi: {acc4:.4f}\n")
print("Classification Report:")
print(classification_report(y_test, pred4, zero_division=0))


=== Percobaan 4: TF-IDF + RandomForest ===
Akurasi: 0.4800

Classification Report:
              precision    recall  f1-score   support

     negatif       0.67      0.50      0.57        20
      netral       0.11      0.20      0.14        10
     positif       0.71      0.60      0.65        20

    accuracy                           0.48        50
   macro avg       0.49      0.43      0.45        50
weighted avg       0.57      0.48      0.52        50

